# PINN #1

## Import Relevant Packages

In [1]:
import pandas as pd
import sciann as sn
import numpy as np
from sklearn.preprocessing import MinMaxScaler

---------------------- SCIANN 0.6.3.1 ---------------------- 
For details, check out our review paper and the documentation at: 
 +  "https://www.sciencedirect.com/science/article/pii/S0045782520307374", 
 +  "https://arxiv.org/abs/2005.08803", 
 +  "https://www.sciann.com". 

 Need support or would like to contribute, please join sciann`s slack group: 
 +  "https://join.slack.com/t/sciann/shared_invite/zt-ne1f5jlx-k_dY8RGo3ZreDXwz0f~CeA" 
 


## Load Data from Data Folder

In [2]:
import glob

df = pd.concat(map(pd.read_csv, glob.glob('data/*.csv')))
df

,Qf,Time,X,Y,Alpha,Temp(degC)
0,500,0.0,0.5,0.0,2.672616e-07,20.009595
1,500,0.0,1.0,0.0,2.672529e-07,20.009028
2,500,0.0,1.5,0.0,2.672558e-07,20.009221
3,500,0.0,2.0,0.0,2.672529e-07,20.009035
4,500,0.0,2.5,0.0,2.672530e-07,20.009039
...,...,...,...,...,...,...
3095,700,15.0,48.0,2.0,9.814811e-01,183.682189
3096,700,15.0,48.5,2.0,9.814803e-01,183.607359
3097,700,15.0,49.0,2.0,9.814794e-01,183.530586
3098,700,15.0,49.5,2.0,9.814784e-01,183.451856


In [3]:
# Shuffle the data so it's easy for the model to learn
df = df.sample(frac=1).reset_index(drop=True)
df

,Qf,Time,X,Y,Alpha,Temp(degC)
0,640,4.5,50.0,0.46,3.432606e-01,74.288036
1,640,12.0,21.0,2.00,9.599310e-01,147.390076
2,580,9.0,38.5,2.00,8.025030e-01,110.925316
3,680,9.5,38.5,2.00,9.061430e-01,131.582045
4,700,9.0,0.0,1.56,8.901466e-01,128.915540
...,...,...,...,...,...,...
170495,700,14.5,49.0,1.00,9.796638e-01,179.695998
170496,640,0.0,2.0,1.00,2.671200e-07,20.000003
170497,620,13.0,16.0,0.00,9.673567e-01,151.694021
170498,600,5.5,38.5,2.00,4.423386e-01,81.125778


In [4]:
pd.set_option("display.max_rows", None, "display.max_columns", None)
df[df.isna().any(axis=1)]

,Qf,Time,X,Y,Alpha,Temp(degC)


## Get only data points where Q = 500

In [5]:
# Get only data points where Qf = 500
df = df.loc[df['Qf'] == 500]

## Seperate Data into Train and Test

In [6]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.2)

In [18]:
train_df.head()

,Qf,Time,X,Y,Alpha,Temp(degC)
98167,500,9.5,0.0,1.8,7.493038e-01,101.363952
117812,500,0.0,45.5,0.0,2.672542e-07,20.009115
94741,500,1.5,48.0,2.0,3.150581e-02,35.083787
145734,500,6.5,46.0,2.0,4.570011e-01,78.846961
147423,500,4.5,1.0,0.0,2.422382e-01,62.570523


## Initialize MinMaxScaler

In [7]:
t_scaler = MinMaxScaler(feature_range = (1e-7,1))
x_scaler = MinMaxScaler(feature_range = (1e-7,1))
y_scaler = MinMaxScaler(feature_range = (1e-7,1))
T_scaler = MinMaxScaler(feature_range = (1e-7,1))
a_scaler = MinMaxScaler(feature_range = (1e-7,1))

 ## Convert input columns into NumPy Arrays and Scale Them

In [8]:
t_np = train_df['Time']
x_np = train_df['X']
y_np = train_df['Y']
alpha_np = train_df['Alpha']
temp_np = train_df['Temp(degC)']

In [9]:
t_np = t_scaler.fit_transform(t_np.to_numpy().reshape(-1,1))
x_np = x_scaler.fit_transform(x_np.to_numpy().reshape(-1,1))
y_np = y_scaler.fit_transform(y_np.to_numpy().reshape(-1,1))
temp_np = T_scaler.fit_transform(temp_np.to_numpy().reshape(-1,1))
alpha_np = a_scaler.fit_transform(alpha_np.to_numpy().reshape(-1,1))

## Define Inputs and Targets for PINN

In [10]:
# Define the necessary inputs and parameter using Variable and Parameter
t = sn.Variable('t')
x = sn.Variable('x')
y = sn.Variable('y')

# Define Neural Network Approximations
T = sn.Functional('T', [t,x,y], 6*[20], 'tanh')
a = sn.Functional('a', [t,x,y], 6*[20], 'tanh')

## Define Constants and PDEs

In [11]:
# Constants

H = 352.1 #1
A = 2e5 #2
E_a = 51100 #3 
n = 1.927 #4
p = 980 #5
c_p = 1440 #6
k = 1.8 #7
R = 8.314 #8
T_o = 20 #9
a_o = 1e-7 #10
h = 1.8 #11
Q = 500

# PDEs
from sciann.utils.math import diff, sign, exp, pow

L1 = (k/(p*c_p))*diff(T, x, order=2) + (k/(p*c_p))*diff(T, y, order=2) + (H/c_p)*diff(a, t) - diff(T, t)
L2 = A*exp((-E_a/(R*T)))*pow(1-a, n)*(1 + 0.36*a) - diff(a, t)

## Define Initial & Boundary Conditions

In [12]:
# t: [0,15]
# x: [0,50]
# y: [0,2]
# Q: [500,700]

tmin = 1e-7
Xmin = 1e-7
Xmax = 1.0
Ymin = 1e-7
Ymax = 1.0

# Boundary Conditions
L3 = (k*diff(T, x))*(1-sign(x - Xmin)) + (h*T)*(1-sign(x - Xmin)) - (h*T_o)*(1-sign(x - Xmin))

L4 = (k*diff(T, x))*(1+sign(x - Xmax)) + (h*T)*(1+sign(x - Xmax)) - (h*T_o)*(1+sign(x - Xmax))

L5 = (k*diff(T, y))*(1-sign(y - Ymin)) + (h*T)*(1-sign(y - Ymin)) - (h*T_o)*(1-sign(y - Ymin))

L6 = (k*diff(T,y) - Q)*(1+sign(y - Ymax))

# Initial Conditions
L7 = (1-sign(t - tmin))*(T_o)

L8 = (1-sign(t - tmin))*(a_o)

L9 = sn.Data(T)

L10 = sn.Data(a)


## Build the Model

In [16]:
m = sn.SciModel([t,x,y], [L1,L2,L3,L4,L5,L6,L7,L8,L9,L10], optimizer="adam",loss_func="mse")

h = m.train(
    [t_np, x_np, y_np], 
    ['zeros','zeros','zeros','zeros','zeros','zeros','zeros','zeros',temp_np,alpha_np], 
    epochs=5000,
    batch_size=64,
    verbose = 1,
    shuffle=False)


Total samples: 12400 
Batch size: 64 
Total batches: 194 

Epoch 1/5000
Batch 0: Invalid loss, terminating training
194/194 [==============================] - 6s 689us/step - batch: 0.0000e+00 - size: 64.0000 - loss: nan - sub_loss: nan - sub_2_loss: nan - sub_6_loss: nan - sub_10_loss: nan - sub_14_loss: nan - mul_25_loss: nan - mul_26_loss: 6.2500 - mul_27_loss: 1.5625e-16 - T_loss: nan - a_loss: nan - lr: 0.0010 - time: 6.1065
